In [569]:
import numpy as np
import matplotlib.pyplot as pyplot
import scipy.stats as sps
from sklearn.metrics import roc_auc_score
# from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
import random
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

[6, 3, 2, 5, 7, 8, 1, 4]


In [570]:
train_data = pd.read_csv("linear_train.txt",names=['x','y'])
test_data = pd.read_csv("linear_test.txt",names=['x'])
test_data = list(test_data['x'])
train_xx = list(train_data['x'])
train_yy = list(train_data['y'])
indices = list(np.arange(len(train_xx)))
indices = random.sample(indices,len(indices))
train_xx = [train_xx[i] for i in indices]
train_yy = [train_yy[i] for i in indices]
C = int(len(train_xx) * 0.6)
train_x = train_xx[0:C]
test_x = train_xx[C:]
train_y = train_yy[0:C]
test_y = train_yy[C:]
# print(train_y)

print(len(test_data))
# test_data = [test_data[i] for i in np.arange(0,len(test_data),100)]

188920


In [459]:
print(sum(y == 1 for y in train_y))
print(sum(y == 0 for y in train_y))

7471
63514


In [571]:
reward = [0.0,2.6,3,5,6,0,0,0,0,0,0]
pen = [0.0,0.1,0.2,0.5,0.5,0,0,0,0,0]
# suf_reward = [1,2]
# suf_pen = [0.1,0.2]
KK = 6
KS = 5
def build_tf_idf(X,Y):
    succes_qnt = dict()
    all_qnt = dict() 
    suff_succes_qnt = dict()
    suff_all_qnt = dict() 
    
    for i in range(len(X)):
        for k in range(0,KK):
            if len(X[i]) > k:
                w = X[i][-k-1:]
                if w not in all_qnt:
                    all_qnt[w] = 0
                all_qnt[w] += 1
                if Y[i] == 1:
                    if w not in succes_qnt:
                        succes_qnt[w] = 0
                    succes_qnt[w] += 1
        for k in range(0,KS):
            if len(X[i]) > k+1:
                w = X[i][-k-2:-1]
                if w not in suff_all_qnt:
                    suff_all_qnt[w] = 0
                suff_all_qnt[w] += 1
                if Y[i] == 1:
                    if w not in suff_succes_qnt:
                        suff_succes_qnt[w] = 0
                    suff_succes_qnt[w] += 1


    tf_idf = dict()
    tf_idf_suf = dict()
    N = np.sum(Y)
    for w in succes_qnt:
#         tf = succes_qnt[w]/N
        tf = abs(succes_qnt[w]/N -  (all_qnt[w] - succes_qnt[w])/(len(Y)))
        idf = len(Y)/all_qnt[w]
        tf_idf[w] = tf * np.log(idf)
    for w in suff_succes_qnt:
#         tf = succes_qnt[w]/N
        tf = abs(suff_succes_qnt[w]/N -  (suff_all_qnt[w] - suff_succes_qnt[w])/(len(Y)))
        idf = len(Y)/suff_all_qnt[w]
        tf_idf_suf[w] = tf * np.log(idf)
    return [tf_idf, tf_idf_suf]

In [577]:
#########FEATURES############
# KS = 2
stat = build_tf_idf(train_x, train_y)
tf_idf = stat[0]
tf_idf_suff = stat[1]

def get_top_from_tfidf(source,length) :
    pairs = []
    for w in source:
        pairs.append((source[w],w))
        
    pairs = list(reversed(sorted(pairs)))
#     for x in pairs[0:20]:
#         print(x)
    pairs = pairs[:min(len(pairs),length)]
    top_ends = set()
    for p in pairs:
        top_ends.add(p[1])
    return top_ends
    
top_ends = get_top_from_tfidf(tf_idf,800)
top_suffs = get_top_from_tfidf(tf_idf_suff,800)
# suf_dicts = build_suff(train_x,train_y)

##################################################
def is_upper_first(w):
    return int(w[0].isupper())

mean_isupper = np.mean([sum(x.isupper() for x in w) for w in train_x])
std_isupper = np.std([sum(x.isupper() for x in w) for w in train_x])
mean_trash = np.mean([len(w) - sum(x.isalpha() for x in w) for w in train_x])
std_trash = np.std([len(w) - sum(x.isalpha() for x in w) for w in train_x])

def number_of_upper(w):
    return (sum(x.isupper() for x in w) - mean_isupper)/std_isupper

def symbol_statistic(w):
    count = dict()
    for x in alp:
        count[x] = 0
    for x in w:
        if x in count :
            count[x] += 1
    return [count[k]/len(w) for k in count]

def num_of_trash(w):
    return (len(w) - sum(x.isalpha() for x in w) - mean_trash)/std_trash

def ending_score_tfidf(w,k):
    if len(w) < k:
        return tf_idf_mean
    x = w[-k-1:]
    if x in tf_idf:
        return tf_idf[x]
    return tf_idf_mean

def ending_score_stupid(w,k):
    if len(w) < k:
        return 0
    x = w[-k-1:]
    if x in my_tf:
        return my_tf[x] / np.sqrt(len(train_x))
    return 0

def suff_binary(w,x):
    if len(w) < len(x) + 1:
        return 0
    if w[-1*len(x)-1:-1] == x:
        return 1
    return 0

def ending_binary_plus(w,x):
    if len(w) < len(x):
        return 0
    if w[-1*len(x):] == x:
        return 1
    return 0

In [409]:
i = 0
print(len(tf_idf))
print(len(top_ends))
for x in top_ends:
    i+= 1
    if i == 4:
        break
    print(x)

22071
20
ер
ова
са


In [578]:
mean_len = np.mean([len(x) for x in train_x])
std_len = np.std([len(x) for x in train_x])
def get_features(w):
    if type(w) != str:
        w = w[0]
    result = []
    result.append(is_upper_first(w))
    result.append((len(w) - mean_len)/std_len)
    result.append(number_of_upper(w))
    result.append(num_of_trash(w))
#     for k in range(1,KK):
#         result.append(ending_score_tfidf(w,k))
#     for k in range(1,KK):
#         result.append(ending_score_stupid(w,k))
    for x in top_ends:
        result.append(ending_binary_plus(w,x))
    for x in top_suffs:
        result.append(suff_binary(w,x))
    return result
def get_features_for_data(data):
    return [get_features(w) for w in data]

In [579]:
regressor = LogisticRegression(penalty='l2',class_weight='balanced',max_iter=300,n_jobs=4)

In [580]:
# print(cross_val_score(regressor,features,train_yy,scoring='roc_auc',n_jobs=4))
features = get_features_for_data(test_x)
print(features[0])
regressor.fit(features,test_y)
y_check_both = regressor.predict_proba(features)
y_check = [y[1] for y in y_check_both]
# print(regressor.score(features,train_y))
print(roc_auc_score(test_y,y_check))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [552]:
features = get_features_for_data(train_x)

In [550]:
regressor.fit(get_features_for_data(train_x),train_y)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=300,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [553]:
y_check_both = regressor.predict_proba(features)
y_check = [y[1] for y in y_check_both]
print(roc_auc_score(train_y,y_check))

0.912569341828


In [554]:
# res = regressor.decision_function(features)
res = regressor.predict_proba(get_features_for_data(test_data))

print(res[0:20])
# print(res2[0:20])
print(train_y[0:100])

[[ 0.09169213  0.90830787]
 [ 0.46182732  0.53817268]
 [ 0.23060508  0.76939492]
 [ 0.49901427  0.50098573]
 [ 0.12117585  0.87882415]
 [ 0.15142828  0.84857172]
 [ 0.22485234  0.77514766]
 [ 0.2946829   0.7053171 ]
 [ 0.2946829   0.7053171 ]
 [ 0.17676626  0.82323374]
 [ 0.17676626  0.82323374]
 [ 0.28562703  0.71437297]
 [ 0.42294444  0.57705556]
 [ 0.22469684  0.77530316]
 [ 0.45466117  0.54533883]
 [ 0.2019476   0.7980524 ]
 [ 0.15137018  0.84862982]
 [ 0.34267385  0.65732615]
 [ 0.04120906  0.95879094]
 [ 0.00708535  0.99291465]]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [555]:
submission = pd.read_csv("linear_ans_example.txt")
submission['Id'] = [i for i in range(len(res))]
submission['Answer'] = res
submission.to_csv("submission_next.txt", sep=',', index=False)
submission.head(10)

,Id,Answer
0,0,0.091692
1,1,0.461827
2,2,0.230605
3,3,0.499014
4,4,0.121176
5,5,0.151428
6,6,0.224852
7,7,0.294683
8,8,0.294683
9,9,0.176766
